## Document Query System using RAG - Llama2 Model With Hugging Face

### INSTALLING & IMPORTING NECESSARY LIBRARIES

In [10]:
!pip install pypdf
!pip install -q transformers einops accelerate langchain bitsandbytes
!pip install sentence_transformers
!pip install llama-index
%pip install llama-index-llms-huggingface
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt
%pip install --upgrade llama_index

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install huggingface_hub
!pip install ipywidgets
!pip install huggingface-hub
!pip install -U bitsandbytes
!pip install -U accelerate
!pip install -U langchain-community
!pip install llama-index --upgrade
%pip install llama-index-embeddings-langchain
!pip install sentence-transformers

from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import Settings

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable


### HUGGING FACE MODEL

In [ ]:
import os
os.environ['HUGGINGFACEHUB_API_TOKEN']= "Your_token"
!huggingface-cli login --token Your_token

'huggingface-cli' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
If you are not able to get the answer from the given document just say
you cannot find the answer dont try to make the answer on your own.
"""
## Default format supportable by LLama2
query_wrapper_prompt=SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

In [5]:
import torch
llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.0, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [6]:
embed_model=LangchainEmbedding(
    HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

Settings.chunk_size=1024
Settings.llm=llm
Settings.embed_model=embed_model

C:\Users\adama\AppData\Local\Temp\ipykernel_6360\2604573702.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))


### FINAL CODE RUN

In [7]:
questionn=input("Enter your Query: ")
print(questionn)

Who is Mr. Oliver?


In [8]:
documents = SimpleDirectoryReader(r"C:\Users\adama\Downloads\RIDS SEM5\data").load_data()
documents
index=VectorStoreIndex.from_documents(documents)
index
query_engine=index.as_query_engine()

In [9]:
response=query_engine.query(questionn)
print(response)

C:\Users\adama\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
C:\Users\adama\AppData\Roaming\Python\Python312\site-packages\transformers\generation\configuration_utils.py:595: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Mr. Oliver is the Anglo-Indian teacher who was returning to his school late one night in the hill station of Simla. He is described as a bachelor who usually strolls into the town in the evening, returning after dark when he takes a short cut through the pine forest. When he encountered a boy sitting alone in the forest, he sensed that something was wrong and asked the boy what was wrong. The boy appeared to be crying and had no face, leading Mr. Oliver to become uneasy and run towards the school buildings.


### RESPONSE OUTPUT

In [12]:
print("Response:",response)

Response: Mr. Oliver is the Anglo-Indian teacher who was returning to his school late one night in the hill station of Simla. He is described as a bachelor who usually strolls into the town in the evening, returning after dark when he takes a short cut through the pine forest. When he encountered a boy sitting alone in the forest, he sensed that something was wrong and asked the boy what was wrong. The boy appeared to be crying and had no face, leading Mr. Oliver to become uneasy and run towards the school buildings.


### EVALUATION METRICS

In [1]:
#What is purchase cost?
!pip install rouge-score

from rouge_score import rouge_scorer

# Define the output and reference texts
output_text = "Purchase cost is the actual price paid for the item. It is independent of the size of the quantity ordered or purchased or manufactured."
reference_text = """Purchase Cost: It is the actual price paid for the item. The cost of purchasing
(producing) one unit of an item is known as purchase cost. It is independent of
the size of the quantity ordered or purchased or manufactured.
Purchase price becomes an important factor when the commodity unit price
becomes dependent on the size of the order. This situation is expressed as
quantity discounts or price breaks."""


# Initialize ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
scores = scorer.score(reference_text, output_text)
print(scores)

#Explain critical acitivity

output_text1 = "A critical activity in project management is an activity in the project network diagram that has a zero total float. This means that it is not affordable to delay the activity without affecting the project's overall timeline. The critical activity is also called the critical path activity, as it lies on the critical path of the project network. The critical path is the sequence of activities that determines the minimum duration required to complete the project."
reference_text1 = """CRITICAL ACTIVITY: An activity which cannot be delayed if the project is to be completed 
on time is known as a critical activity. 
Total float for every critical activity is zero."""
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
scores1 = scorer.score(reference_text1, output_text1)
print(scores1)

#State rules of drawing network?
output_text2="""1. Each activity is represented by only one arrow in the network.
2. The arrows should be drawn as straight lines.
3. The arrows should not cross each other.
4. The length of an arrow is of no significance.
5. Time flows from left to right.
6. Arrows pointing in opposite directions are to be avoided.
7. Angles between the arrows should be as large as possible.
8. Every network should have a unique start event.
9. Every network should have a unique end event.
10. Before an activity can be undertaken, all activities preceding it must be completed.
11. There should be no duplication in event numbers.
12. No two activities can have the same start event and the same end event.
13. No loops are allowed in the network.
 """
reference_text2 = """1. Each activity is represented by only one arrow in the network.
2. The arrows should be drawn as straight lines.
3. The arrows should not cross each other.
4. The length of an arrow is of no significance.
5. Time flows from left to right. 
6. Arrows pointing in opposite directions are to be avoided.
7. Angles between the arrows should be as large as possible.
8. Every network should have a unique start event.
9. Every network should have a unique end event.
10. Before an activity can be undertaken, all activities preceding it must be completed. 
11. There should be no duplication in event numbers
12.No two activities can have same start event and same end event. Such a situation if 
exists can be avoided by introducing a dummy activity in the network.
13. No loops are allowed in the network.
"""
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
scores2 = scorer.score(reference_text2, output_text2)
print(scores2)

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24972 sha256=2dadb8ccf2ccc41ddec4fe497ff84316769c1d75f84473b826a139bd6cb877f1
  Stored in directory: c:\users\adama\appdata\local\pip\cache\wheels\85\9d\af\01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score
{'rouge1': Score(precision=1.0, recall=0.34782608695652173, fmeasure=0.5161290322580645), 'rouge2': Score(precision=0.9130434782608695, recall=0.3088235294117647, fmeasure=0.46153846153846156), 'rougeL': Score(precision=1.0, recall=0.34782608695652173, fmeasure=0.5161290322580645)}
{'rouge1': Score(precision=0.2857142857142857, recall=0.7096774193548387, fmeasure=0.40740740740740744), 'rouge2': Score(precision=0.09210526315789473, recall=0.23333333333333334, fmeasure=0.13207547169811318), 'rouge